In [2]:
import numpy as np
import pandas as pd
import pickle
import os

os.environ["CUDA_VISIBLE_DEVICES"]= '0' #set gpu 

### Load Data and Functions

In [3]:
#Load Auxiliary functions
exec(open('fcd/FCD.py').read()) #frechet distance, keras loss functions, data_generator, CHEMBLNET

# Load generated molecules
gen_mol_file = "generated_smiles/LSTM_Segler.smi" #input file which contains one generated SMILES per line
gen_mol = pd.read_csv(gen_mol_file,header=None)[0] #IMPORTANT: take at least 10000 molecules as FCD can vary with sample size 
sample = np.random.choice(gen_mol, 10000, replace=False)
sample_std = canoncial_smiles(sample)
 #IMPORTANT: use canoncial SMILES otherwise the FCD might be not representative
# Load ChEMNet and reference stats
mu_ref, cov_ref = pickle.load(open("fcd/ref_combined_stat.p", 'rb')).values() #statistics of random 50,000 chembl molecules, which were not used for training
model = load_ref_model('fcd/ChemNet_v0.13_pretrained.h5')

Using TensorFlow backend.
/home/lfr/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lfr/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lfr/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lfr/anaconda3/lib/python3.7/site-packages/tensorflow/python

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



## Calculation of Frechet CHEMBLNET DISTANCE

In [ ]:
#get CHEBMLNET activations of generated molecules 
gen_mol_act = get_predictions(model, sample_std)
FCD = calculate_frechet_distance(mu1=np.mean(gen_mol_act, axis=0), mu2=mu_ref, 
                                 sigma1=np.cov(gen_mol_act.T), sigma2=cov_ref)
FCD